<a href="https://colab.research.google.com/github/Lawrence-Krukrubo/SQL_for_Data_Science/blob/main/sql_for_data_analysis1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b><h1>Welcome To SQL Basics...</h1></b>

We connect to Google CloudSQL and make analysis with the Patch and Posey Database.<br>

Thanks to this [article](https://towardsdatascience.com/sql-on-the-cloud-with-python-c08a30807661) for making the connection process clearer.

If we want to download the parch-and-posey.sql file to maybe upload to a database, use this [link](https://storage.googleapis.com/kaggle1980/parch.sql) to the updated file from cloud-storage.

In [1]:
# Next mount gdrive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# set working directory to Udacity
%cd /content/gdrive/MyDrive/Colab_Notebooks/Udacity

/content/gdrive/MyDrive/Colab_Notebooks/Udacity


In [3]:
%ls

 aws_machine_learning_foundations/   linear_algebra_refresher.ipynb
 client-cert.pem                    'linear-example-data (1).xlsx'
 client-key.pem                      Problem_Solving_w_Advanced_Analytics.ipynb
 computer_vision/                    server-ca.pem
 intro_to_algorithm.ipynb            statistics/
 intro_to_artificial_intelligence/   time_series_forecasting.ipynb
 intro_to_data_analysis/             Udac_Prog_Foundations_Python/
 intro_to_machine_learning.ipynb     version_control_with_git/


In [4]:
!pip install mysql-connector-python

     |████████████████████████████████| 18.1MB 173kB/s 


In [13]:
import mysql.connector
from mysql.connector.constants import ClientFlag
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
import time

In [6]:
config = {
    'user': 'root',
    'password': 'root',
    'host': '35.226.26.66',
    'client_flags': [ClientFlag.SSL],
    'ssl_ca': 'server-ca.pem',
    'ssl_cert': 'client-cert.pem',
    'ssl_key': 'client-key.pem'
}

# now we establish our connection
try:
    cnxn = mysql.connector.connect(**config)
    print('Connection to CloudSQL Instance Successful!')
except Exception as e:
    print(e)

Connection to CloudSQL Instance Successful!


In [7]:
config

{'client_flags': [2048],
 'host': '35.226.26.66',
 'password': 'root',
 'ssl_ca': 'server-ca.pem',
 'ssl_cert': 'client-cert.pem',
 'ssl_key': 'client-key.pem',
 'user': 'root'}

In [8]:
# let's add the patch and posey db to our config

# cursor = cnxn.cursor()  # initialize connection cursor
# cursor.execute('CREATE DATABASE parch_and_posey_db')  # create a new database
# cnxn.close()  # close connection because we will be reconnecting to testdb

Now we connect to parch_and_posey_db by adding database: parch_and_posey_db to our config dictionary and connecting just like we did before:

In [9]:
config['database'] = 'parch_and_posey_db'  # add new database to config dict
cnxn = mysql.connector.connect(**config)
cursor = cnxn.cursor()

Let's see the first 3 data of the different tables in parch and posey database

In [10]:
# let's run the show tables command 

cursor.execute('show tables')
out = cursor.fetchall()
out

[('accounts',), ('orders',), ('region',), ('sales_reps',), ('web_events',)]

let's see some of the data in our tables

Defining a method that converts a select query to a data frame

In [11]:
def query_to_df(query):
    st = time.time()
    # Assert Every Query ends with a semi-colon
    try:
        assert query.endswith(';')
    except AssertionError:
        return 'ERROR: Query Must End with ;'

    # so we never have more than 20 rows displayed
    pd.set_option('display.max_rows', 20) 
    df = None

    # Process the query
    cursor.execute(query)
    columns = cursor.description
    result = []
    for value in cursor.fetchall():
        tmp = {}
        for (index,column) in enumerate(value):
            tmp[columns[index][0]] = [column]
        result.append(tmp)

    # Create a DataFrame from all results
    for ind, data in enumerate(result):
        if ind >= 1:
            x = pd.DataFrame(data)
            df = pd.concat([df, x], ignore_index=True)
        else:
            df = pd.DataFrame(data)
    print(f'Query ran for {time.time()-st} secs!')
    return df

In [14]:
# 1. For the accounts table
query = 'SELECT * FROM accounts LIMIT 3;'
query_to_df(query)

Query ran for 0.054364681243896484 secs!


,id,name,website,lats,longs,primary_poc,sales_rep_id
0,1001,Walmart,www.walmart.com,40.238496,-75.103297,Tamara Tuma,321500
1,1011,Exxon Mobil,www.exxonmobil.com,41.169156,-73.849374,Sung Shields,321510
2,1021,Apple,www.apple.com,42.290495,-76.084009,Jodee Lupo,321520


In [15]:
# 2. For the orders table

query = 'SELECT * FROM orders LIMIT 3;'
query_to_df(query)

Query ran for 0.04391670227050781 secs!


,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,1,1001,2015-10-06 17:31:14,123,22,24,169,613.77,164.78,194.88,973.43
1,2,1001,2015-11-05 03:34:33,190,41,57,288,948.10,307.09,462.84,1718.03
2,3,1001,2015-12-04 04:21:55,85,47,0,132,424.15,352.03,0.00,776.18


In [16]:
# 3. For the region table

query = 'SELECT * FROM region LIMIT 3;'
query_to_df(query)

Query ran for 0.036257266998291016 secs!


,id,name
0,1,Northeast
1,2,Midwest
2,3,Southeast


In [17]:
# 4. For the sales_reps table

query = 'SELECT * FROM sales_reps LIMIT 3;'
query_to_df(query)

Query ran for 0.038065433502197266 secs!


,id,name,region_id
0,321500,Samuel Racine,1
1,321510,Eugena Esser,1
2,321520,Michel Averette,1


In [18]:
# 5. For the web_events table

query = 'SELECT * FROM web_events LIMIT 3;'
query_to_df(query)

Query ran for 0.037531375885009766 secs!


,id,account_id,occurred_at,channel
0,1,1001,2015-10-06 17:13:58,direct
1,2,1001,2015-11-05 03:08:26,direct
2,3,1001,2015-12-04 03:57:24,direct


In [19]:
# let's close the cursor connection after running these queries
cursor.close()

True

The SQL language has a few different elements. The most basic of which is the `statement`. Think of a statement as a piece of correctly written SQL codes. Statements tell the database what you'd like to do with the data.<br>For example `drop`, `create`, `alter` and so on. The most common tho is the `select` statement, which allows us to read data and display em. `Select` statements are commonly referrred to as queries.<br>
Both the `select` and `from` clauses are mandatory, included in any query.

The `SELECT` statement is where you put the columns for which you would like to show the data. The `FROM` statement is where you put the tables from which you would like to pull data.

In [20]:
# We closed the cursor connection above, so we reopen it first
cursor = cnxn.cursor()

Now let's select 10 rows from just a few columns from the orders table

In [21]:
query = 'SELECT id, account_id, occurred_at FROM orders LIMIT 10;'
query_to_df(query)

Query ran for 0.052858829498291016 secs!


,id,account_id,occurred_at
0,1,1001,2015-10-06 17:31:14
1,2,1001,2015-11-05 03:34:33
2,3,1001,2015-12-04 04:21:55
3,4,1001,2016-01-02 01:18:24
4,5,1001,2016-02-01 19:27:27
5,6,1001,2016-03-02 15:29:32
6,7,1001,2016-04-01 11:20:18
7,8,1001,2016-05-01 15:55:51
8,9,1001,2016-05-31 21:22:48
9,10,1001,2016-06-30 12:32:05


<h3>Formatting Best Practices..</h3>

1. **Using Upper and Lower Case in SQL:**<br>
SQL queries can be run successfully whether characters are written in upper- or lower-case. In other words, SQL queries are not case-sensitive

2. **Capitalizing SQL Clauses:**<br>It is common and best practice to capitalize all SQL commands, like `SELECT` and `FROM`, and keep everything else in your query lower case. Capitalizing command words makes queries easier to read, which will matter more as you write more complex queries.

3. **One other note:**<BR> The text data stored in SQL tables can be either upper or lower case, and SQL is case-sensitive in regard to this text data.

4. **Avoid Spaces in Table and Variable Names:**<br>
It is common to use underscores and avoid spaces in column names. It is a bit annoying to work with spaces in SQL. In Postgres if you have spaces in column or table names, you need to refer to these columns/tables with double quotes around them (Ex: `FROM "Table Name"` as opposed to `FROM table_name`). In other environments, you might see this as square brackets instead (Ex: `FROM [Table Name]`).

5. **Use White Space in Queries:**<br>
SQL queries ignore spaces, so you can add as many spaces and blank lines between code as you want, and the queries are the same. But pls use with decorum.

6. **Semicolons:**<br>
Depending on your SQL environment, your query may need a semicolon at the end to execute. Other environments are more flexible in terms of this being a "requirement." It is considered best practice to put a semicolon at the end of each statement, which also allows you to run multiple queries at once if your environment allows this.

**The LIMIT clause**

* The `LIMIT` command is always the very last part of a query.

**The ORDER BY Clause**

* The `ORDER BY` statement allows us to sort our results using the data in any column. 
* Using `ORDER BY` in a SQL query only has temporary effects, for the results of that query, unlike sorting a sheet by column in Excel or Sheets.
* The `ORDER BY` statement always comes in a query after the `SELECT` and `FROM` statements, but before the `LIMIT` statement. If you are using the `LIMIT` statement, it will always appear last.
* Remember **DESC** can be added after the column in your `ORDER BY` statement to sort in descending order, as the default is to sort in ascending order.

Write a query to return the 10 earliest orders in the orders table. Include the `id`, `occurred_at`, and `total_amt_usd` columns.

In [22]:
query = 'SELECT id, occurred_at, total_amt_usd FROM orders \
        ORDER BY occurred_at LIMIT 10;'
query_to_df(query)

Query ran for 0.053076744079589844 secs!


,id,occurred_at,total_amt_usd
0,5786,2013-12-04 04:22:44,627.48
1,2415,2013-12-04 04:45:54,2646.77
2,4108,2013-12-04 04:53:25,2709.62
3,4489,2013-12-05 20:29:16,277.13
4,287,2013-12-05 20:33:56,3001.85
5,1946,2013-12-06 02:13:20,2802.90
6,6197,2013-12-06 12:55:22,7009.18
7,3122,2013-12-06 12:57:41,1992.13
8,6078,2013-12-06 13:14:47,6680.06
9,2932,2013-12-06 13:17:25,2075.94


Write a query to return the top 5 orders in terms of largest `total_amt_usd`. Include the `id`, `account_id`, and `total_amt_usd`.

In [23]:
query = 'SELECT id, account_id, total_amt_usd FROM orders \
        ORDER BY total_amt_usd DESC LIMIT 5;'
query_to_df(query)

Query ran for 0.04217362403869629 secs!


,id,account_id,total_amt_usd
0,4016,4251,232207.06
1,3892,4161,112875.18
2,3963,4211,107533.55
3,5791,2861,95005.82
4,3778,4101,93547.84


Write a query to return the lowest 20 orders in terms of smallest `total_amt_usd`. <br>Include the `id`, `account_id`, and `total_amt_usd`.

In [24]:
query = 'SELECT id, account_id, total_amt_usd FROM orders ORDER \
        BY total_amt_usd LIMIT 20;'
query_to_df(query)

Query ran for 0.06483292579650879 secs!


,id,account_id,total_amt_usd
0,2518,2881,0.0
1,4844,1571,0.0
2,5057,1851,0.0
3,4490,1281,0.0
4,6281,3491,0.0
5,6323,3551,0.0
6,4770,1521,0.0
7,6312,3541,0.0
8,5001,1791,0.0
9,6435,3801,0.0


**The ORDER BY 2 Clause**

* We can `ORDER BY` more than one column at a time
* When you provide a list of columns in an `ORDER BY` command, the sorting occurs using the leftmost column in your list first, then the next column from the left, and so on.
* We still have the ability to flip the way we order using `DESC`.

Write a query that displays the order `ID`, `account ID`, and `total dollar amount` for all the orders, <br>sorted first by the `account ID` (in ascending order), and then by the `total dollar amount` (in descending order).


In [25]:
query = 'SELECT id, account_id, total_amt_usd FROM orders \
        ORDER BY account_id, total_amt_usd DESC;'
query_to_df(query)

Query ran for 7.764667510986328 secs!


,id,account_id,total_amt_usd
0,4308,1001,9426.71
1,4309,1001,9230.67
2,4316,1001,9134.31
3,4317,1001,8963.91
4,4314,1001,8863.24
...,...,...,...
6907,4301,4501,974.17
6908,4300,4501,875.54
6909,4306,4501,628.74
6910,4299,4501,157.24


Now write a query that again displays `order ID`, `account ID`, and `total dollar amount` for each order, but this time sorted first by `total dollar amount` (in descending order), and then by `account ID` (in ascending order).

In [26]:
query = 'SELECT id, account_id, total_amt_usd FROM orders \
        ORDER BY total_amt_usd DESC, account_id;'
query_to_df(query)

Query ran for 7.715097188949585 secs!


,id,account_id,total_amt_usd
0,4016,4251,232207.06
1,3892,4161,112875.18
2,3963,4211,107533.55
3,5791,2861,95005.82
4,3778,4101,93547.84
...,...,...,...
6907,6323,3551,0.00
6908,6375,3651,0.00
6909,6435,3801,0.00
6910,6523,3991,0.00


Compare the results of these two queries above. How are the results different when you switch the column you sort on first?

**Ans:** 
* For the first query, order by account_id first ensures that each account_id ascends from lowest to highest. And for each account_id, the total_amt_usd is decreasing from highest to lowest.
* For the second query where we sort by total_amt_usd DESC first, the query is sorted by the observations with the highest total_amt_usd first, and the account_id next. Now if each account_id has multiple total_amt_usd, then these would be sorted by total_amt_usd Desc first.
* In query #1, all of the orders for each account ID are grouped together, and then within each of those groupings, the orders appear from the greatest order amount to the least. 
* In query #2, since you sorted by the total dollar amount first, the orders appear from greatest to least regardless of which account ID they were from. Then they are sorted by account ID next. (The secondary sorting by account ID is difficult to see here, since only if there were two orders with equal total dollar amounts would there need to be any sorting by account ID.)



**The WHERE Clause:**

* Using the `WHERE` statement, we can display subsets of tables based on conditions that must be met. You can also think of the `WHERE` command as filtering the data.
* The `WHERE` clause goes after `FROM`, but before `ORDER BY` or `LIMIT`
* Common symbols used in `WHERE` statements include:
** $>$ (greater than)
** $<$ (less than)
** $>=$ (greater than or equal to)
** $<=$ (less than or equal to)
** $=$ (equal to)
** $!=$ (not equal to)

Write a query that:

Pulls the first 5 rows and all columns from the orders table that have a dollar amount of gloss_amt_usd greater than or equal to 1000.

In [27]:
query = 'SELECT * FROM orders WHERE gloss_amt_usd >= 1000 LIMIT 5;'
query_to_df(query)

Query ran for 0.04915308952331543 secs!


,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,14,1001,2016-10-26 20:31:30,97,143,54,294,484.03,1071.07,438.48,1993.58
1,62,1091,2014-10-13 12:12:55,146,196,3,345,728.54,1468.04,24.36,2220.94
2,88,1101,2015-06-24 13:08:15,182,339,17,538,908.18,2539.11,138.04,3585.33
3,121,1131,2016-08-10 23:47:41,273,134,0,407,1362.27,1003.66,0.00,2365.93
4,129,1141,2016-12-21 15:52:58,143,1045,2157,3345,713.57,7827.05,17514.84,26055.46


Pulls the first 10 rows and all columns from the orders table that have a total_amt_usd less than 500.

In [28]:
query = 'SELECT * FROM orders WHERE total_amt_usd < 500  LIMIT 10;'
query_to_df(query)

Query ran for 0.059696197509765625 secs!


,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,67,1091,2015-04-07 13:29:20,95,0,0,95,474.05,0.00,0.00,474.05
1,96,1101,2016-03-15 11:36:03,14,8,16,38,69.86,59.92,129.92,259.70
2,119,1131,2016-06-12 12:29:45,0,30,23,53,0.00,224.70,186.76,411.46
3,124,1131,2016-11-07 05:10:56,0,0,0,0,0.00,0.00,0.00,0.00
4,254,1251,2014-11-01 02:15:24,0,0,17,17,0.00,0.00,138.04,138.04
5,328,1291,2015-08-03 08:35:23,0,19,21,40,0.00,142.31,170.52,312.83
6,542,1421,2015-11-13 09:07:09,0,64,0,64,0.00,479.36,0.00,479.36
7,683,1501,2016-04-14 23:59:50,0,15,16,31,0.00,112.35,129.92,242.27
8,713,1521,2014-11-23 16:04:03,0,8,10,18,0.00,59.92,81.20,141.12
9,730,1521,2016-05-06 02:34:48,0,0,2,2,0.00,0.00,16.24,16.24


**WHERE Clause contd...**

The `WHERE` statement can also be used with non-numeric data. We can use the = and != operators here. You need to be sure to use single quotes (just be careful if you have quotes in the original text) with the text data, not double quotes.

Commonly when we are using `WHERE` with non-numeric data fields, we use the `LIKE`, `NOT`, or `IN` operators. 

Filter the accounts table to include the company name, website, and the primary point of contact (primary_poc) just for the Exxon Mobil company in the accounts table.

In [29]:
query = "SELECT name, website, primary_poc FROM accounts WHERE name = 'Exxon Mobil';"
query_to_df(query)

Query ran for 0.03558039665222168 secs!


,name,website,primary_poc
0,Exxon Mobil,www.exxonmobil.com,Sung Shields


Note: If you received an error message when executing your query, remember that SQL requires single-quotes, not double-quotes, around text values like 'Exxon Mobil.'

**Derived Columns:**

* Creating a new column that is a combination of existing columns is known as a derived column (or "calculated" or "computed" column). Usually you want to give a name, or "alias," to your new column using the `AS` keyword.
* This derived column, and its alias, are generally only temporary, existing just for the duration of your query. The next time you run a query and access this table, the new column will not be there.
* **Order of Operations**<br>
Remember **PEMDAS** from math class to help remember the order of operations? The same order of operations applies when using arithmetic operators in SQL.

The following two statements have very different end results:

* Standard_qty / standard_qty + gloss_qty + poster_qty
* standard_qty / (standard_qty + gloss_qty + poster_qty)

Select id, account_id, gloss_amt_usd, total_amt_usd and add a derived column called pct_gloss_amt by dividing gloss_amt_usd by total_amt_usd. Show only 10 rows

In [30]:
query = 'SELECT id, account_id, gloss_amt_usd, total_amt_usd, \
        (gloss_amt_usd/total_amt_usd) AS pct_gloss_amt FROM orders LIMIT 10;'
query_to_df(query)

Query ran for 0.04852128028869629 secs!


,id,account_id,gloss_amt_usd,total_amt_usd,pct_gloss_amt
0,1,1001,164.78,973.43,0.169278
1,2,1001,307.09,1718.03,0.178745
2,3,1001,352.03,776.18,0.453542
3,4,1001,239.68,958.24,0.250125
4,5,1001,217.21,983.49,0.220856
5,6,1001,179.76,1067.25,0.168433
6,7,1001,247.17,1498.20,0.164978
7,8,1001,352.03,2052.20,0.171538
8,9,1001,119.84,752.57,0.159241
9,10,1001,344.54,878.56,0.392164


Using the orders table Create a column that divides the standard_amt_usd by the standard_qty to find the unit price for standard paper for each order. Limit the results to the first 10 orders, and include the id and account_id fields.


In [31]:
query = 'SELECT id, account_id, (standard_amt_usd / standard_qty) \
        AS unit_price FROM orders LIMIT 10;'

query_to_df(query)

Query ran for 0.04488873481750488 secs!


,id,account_id,unit_price
0,1,1001,4.99
1,2,1001,4.99
2,3,1001,4.99
3,4,1001,4.99
4,5,1001,4.99
5,6,1001,4.99
6,7,1001,4.99
7,8,1001,4.99
8,9,1001,4.99
9,10,1001,4.99


Using the orders table Write a query that finds the percentage of revenue that comes from poster paper for each order. You will need to use only the columns that end with _usd. (Try to do this without using the total column.) Display the id and account_id fields also. NOTE - you will receive an error with the correct solution to this question. This occurs because at least one of the values in the data creates a division by zero in your formula. You will learn later in the course how to fully handle this issue. For now, you can just limit your calculations to the first 10 orders, as we did in question #1, and you'll avoid that set of data that causes the problem.

In [32]:
query = 'SELECT id, account_id, \
        (poster_amt_usd / (poster_amt_usd + gloss_amt_usd + standard_amt_usd)) \
        AS pct_revenue_poster FROM orders LIMIT 10;'
query_to_df(query)

Query ran for 0.04524374008178711 secs!


,id,account_id,pct_revenue_poster
0,1,1001,0.200199
1,2,1001,0.269402
2,3,1001,0.000000
3,4,1001,0.000000
4,5,1001,0.231177
5,6,1001,0.349984
6,7,1001,0.498625
7,8,1001,0.597466
8,9,1001,0.237373
9,10,1001,0.073939


********************************************************************************

<h3>Introduction to Logical Operators</h3>

In the next concepts, you will be learning about Logical Operators. Logical Operators include:

* **LIKE**<br>
This allows you to perform operations similar to using `WHERE` and `=`, but for cases when you might not know exactly what you are looking for.

* **IN**<br>
This allows you to perform operations similar to using `WHERE` and `=`, but for more than one condition.

* **NOT**<br>
This is used with `IN` and `LIKE` to select all of the rows `NOT LIKE` or `NOT IN` a certain condition.

* **AND & BETWEEN**<br>
These allow you to combine operations where all combined conditions must be true.

* **OR**<br>
This allow you to combine operations where at least one of the combined conditions must be true.

**The LIKE Operator:**

The `LIKE` operator is extremely useful for working with text. You will use `LIKE` within a `WHERE` clause. The `LIKE` operator is frequently used with `%`. The `%` tells us that we might want any number of characters leading up to a particular set of characters or following a certain set of characters.

Remember to use single quotes for the text you pass to the `LIKE` operator, because of this lower and uppercase letters are not the same within the string. Searching for 'T' is not the same as searching for 't'.

**The IN Operator:**

The `IN` operator is useful for working with both numeric and text columns. This operator allows you to use an `=`, but for more than one item of that particular column. We can check one, two or many column values for which we want to pull data, but all within the same query.

**The NOT Operator:**

The `NOT` operator is an extremely useful operator for working with the previous two operators we introduced: `IN` and `LIKE`. By specifying `NOT` `LIKE` or `NOT` `IN`, we can grab all of the rows that do not meet a particular criteria.

**Expert Tip**

In most SQL environments, although not in our Udacity's classroom, you can use single or double quotation marks - and you may NEED to use double quotation marks if you have an apostrophe within the text you are attempting to pull.

Questions using LIKE Operator

Using the accounts table find All the companies whose names start with 'C'.

In [33]:
query = "SELECT * FROM accounts WHERE name LIKE 'C%';"
query_to_df(query)

Query ran for 0.1345834732055664 secs!


,id,name,website,lats,longs,primary_poc,sales_rep_id
0,1061,CVS Health,www.cvshealth.com,41.467796,-73.767636,Anabel Haskell,321560
1,1131,Chevron,www.chevron.com,42.611941,-76.361231,Paige Bartos,321630
2,1141,Costco,www.costco.com,42.263046,-74.809169,Dominique Favela,321640
3,1201,Cardinal Health,www.cardinal.com,41.319065,-77.525122,Deanne Hertlein,321700
4,1281,Citigroup,www.citigroup.com,40.853054,-76.490448,Bea Sera,321570
...,...,...,...,...,...,...,...
32,4121,Corning,www.corning.com,38.404186,-121.435528,Chad Winer,321920
33,4161,Core-Mark Holding,www.core-mark.com,38.604882,-121.399458,Marita Demasi,321960
34,4181,Cameron International,www.c-a-m.com,38.656831,-121.934032,Antwan Seagraves,321980
35,4201,Crown Holdings,www.crowncork.com,38.515206,-121.169341,Jung Soden,321930


All companies whose names contain the string 'one' somewhere in the name.

In [34]:
query = "SELECT * FROM accounts WHERE name LIKE '%one%';"
query_to_df(query)

Query ran for 0.04733872413635254 secs!


,id,name,website,lats,longs,primary_poc,sales_rep_id
0,1741,Honeywell International,www.honeywell.com,40.751170,-73.981919,Jeanne Harty,321700
1,1821,INTL FCStone,www.intlfcstone.com,42.359985,-71.063805,Ericka Hamid,321630
2,2111,Capital One Financial,www.capitalone.com,41.878116,-87.632640,Debby Myers,321760
3,3791,AutoZone,www.autozone.com,34.058263,-118.237747,Maryellen Strohmeyer,321990
4,4471,Oneok,www.oneok.com,45.513513,-122.681500,Esta Engelhardt,321960


All companies whose names end with 's'

In [35]:
query = "SELECT * FROM accounts WHERE name LIKE '%s';"
query_to_df(query)

Query ran for 0.2673625946044922 secs!


,id,name,website,lats,longs,primary_poc,sales_rep_id
0,1071,General Motors,www.gm.com,40.805518,-76.710181,Barrie Omeara,321570
1,1441,United Technologies,www.utc.com,42.329397,-75.157855,Janett Wisecarver,321520
2,1461,Lowe's,www.lowes.com,42.023052,-74.822106,Roselee Pardini,321540
3,1471,UPS,www.ups.com,41.878838,-73.849300,Cedrick Weiss,321550
4,1531,Cisco Systems,www.cisco.com,41.201011,-76.538247,Deadra Waggener,321610
...,...,...,...,...,...,...,...
76,4341,United Natural Foods,www.unfi.com,36.170110,-115.147136,Savanna Gayman,321920
77,4351,Dean Foods,www.deanfoods.com,36.176213,-115.139491,Anabel Haskell,321910
78,4371,Mohawk Industries,www.mohawkind.com,36.180328,-115.135964,Kym Hagerman,321910
79,4431,Franklin Resources,www.franklinresources.com,45.533894,-122.682215,Dominique Favela,321970


Questions using IN operator<br>
Use the accounts table to find the account name, primary_poc, and sales_rep_id for Walmart, Target, and Nordstrom.

In [36]:
query = "SELECT name, primary_poc, sales_rep_id FROM accounts \
        WHERE name IN ('Walmart', 'Target', 'Nordstrom');"
query_to_df(query)

Query ran for 0.038263559341430664 secs!


,name,primary_poc,sales_rep_id
0,Walmart,Tamara Tuma,321500
1,Target,Luba Streett,321660
2,Nordstrom,Yan Crater,321820


Use the web_events table to find all information regarding individuals who were contacted via the channel of organic or adwords.

In [37]:
query = 'SELECT * FROM web_events WHERE channel IN ("organic", "adwords");'
query_to_df(query)

Query ran for 3.1691787242889404 secs!


,id,account_id,occurred_at,channel
0,4395,1001,2015-10-22 05:02:47,organic
1,4396,1001,2015-10-22 14:04:20,adwords
2,4399,1001,2016-01-01 15:45:54,adwords
3,4401,1001,2016-02-07 17:44:10,adwords
4,4402,1001,2016-02-27 15:27:22,organic
...,...,...,...,...
1853,9064,4491,2016-05-06 05:03:27,adwords
1854,9065,4491,2016-06-17 19:43:12,adwords
1855,9071,4491,2016-11-06 07:23:45,organic
1856,9072,4491,2016-12-18 03:21:31,organic


Questions using the NOT operator:

Use the accounts table to find the account name, primary poc, and sales rep id for all stores except Walmart, Target, and Nordstrom.


In [38]:
query = "SELECT name, primary_poc, sales_rep_id FROM accounts \
        WHERE name NOT IN ('Walmart', 'Target', 'Nordstrom');"
query_to_df(query)

Query ran for 0.4585721492767334 secs!


,name,primary_poc,sales_rep_id
0,Exxon Mobil,Sung Shields,321510
1,Apple,Jodee Lupo,321520
2,Berkshire Hathaway,Serafina Banda,321530
3,McKesson,Angeles Crusoe,321540
4,UnitedHealth Group,Savanna Gayman,321550
...,...,...,...
343,KKR,Buffy Azure,321970
344,Oneok,Esta Engelhardt,321960
345,Newmont Mining,Khadijah Riemann,321970
346,PPL,Deanne Hertlein,321960


Use the web_events table to find all information regarding individuals who were contacted via any method except using organic or adwords methods.

In [39]:
query = 'SELECT * FROM web_events WHERE channel NOT IN ("organic", "adwords");'
query_to_df(query)

Query ran for 12.316229820251465 secs!


,id,account_id,occurred_at,channel
0,1,1001,2015-10-06 17:13:58,direct
1,2,1001,2015-11-05 03:08:26,direct
2,3,1001,2015-12-04 03:57:24,direct
3,4,1001,2016-01-02 00:55:03,direct
4,5,1001,2016-02-01 19:02:33,direct
...,...,...,...,...
7210,9066,4491,2016-07-18 21:31:10,facebook
7211,9067,4491,2016-08-14 15:41:44,facebook
7212,9068,4491,2016-09-11 19:18:07,banner
7213,9069,4491,2016-10-04 15:43:29,facebook


Use the accounts table to find, All the companies whose names do not start with 'C'.

In [40]:
query = 'SELECT name FROM accounts WHERE name NOT LIKE "C%";'
query_to_df(query)

Query ran for 0.2737278938293457 secs!


,name
0,Walmart
1,Exxon Mobil
2,Apple
3,Berkshire Hathaway
4,McKesson
...,...
309,KKR
310,Oneok
311,Newmont Mining
312,PPL


All companies whose names do not contain the string 'one' somewhere in the name.

In [41]:
query = 'SELECT name FROM accounts WHERE name NOT LIKE "%one%";'
query_to_df(query)

Query ran for 0.2999308109283447 secs!


,name
0,Walmart
1,Exxon Mobil
2,Apple
3,Berkshire Hathaway
4,McKesson
...,...
341,Lithia Motors
342,KKR
343,Newmont Mining
344,PPL


All companies whose names do not end with 's'.

In [42]:
query = 'SELECT name FROM accounts WHERE name NOT LIKE "%s";'
query_to_df(query)

Query ran for 0.24939560890197754 secs!


,name
0,Walmart
1,Exxon Mobil
2,Apple
3,Berkshire Hathaway
4,McKesson
...,...
265,KKR
266,Oneok
267,Newmont Mining
268,PPL


**The AND Operator:**

* The `AND` operator is used within a `WHERE` statement to consider more than one logical clause at a time
* Each time you link a new statement with an `AND`, you will need to specify the column you are interested in looking at. You may link as many statements as you would like to consider at the same time.
* This operator works with all of the operations we have seen so far including arithmetic operators `(+, *, -, /)`. `LIKE, IN`, and `NOT` logic can also be linked together using the `AND` operator.


**The BETWEEN Operator:**

* Sometimes we can make a cleaner statement using `BETWEEN` than we can using `AND`. Particularly this is true when we are using the same column for different parts of our `AND` statement.
* Note that the endpoints of a `BETWEEN` operator query are inclusive. both start and end limits, included in the output.

For example, statement 1 below is much better written as statement 2 below.

1. `SELECT * FROM table WHERE column >= 6 AND column <= 10`
2. `SELECT * FROM table WHERE column BETWEEN 6 AND 10`

<h2>Questions using AND and BETWEEN operators</h2>

Write a query that returns all the orders where the standard_qty is over 1000, the poster_qty is 0, and the gloss_qty is 0.

In [43]:
query = 'SELECT * FROM orders WHERE standard_qty > 1000 \
        AND poster_qty = 0 AND gloss_qty = 0;'
query_to_df(query)

Query ran for 0.04233813285827637 secs!


,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,2613,2951,2016-08-15 00:06:12,1171,0,0,1171,5843.29,0.0,0.0,5843.29
1,3260,3491,2014-08-29 22:43:00,1552,0,0,1552,7744.48,0.0,0.0,7744.48


Using the accounts table, find all the companies whose names do not start with 'C' and end with 's'.

In [44]:
query = 'SELECT name FROM accounts WHERE name NOT LIKE "C%" AND name LIKE "%S";'
query_to_df(query)

Query ran for 0.08780336380004883 secs!


,name
0,General Motors
1,United Technologies
2,Lowe's
3,UPS
4,HCA Holdings
...,...
64,United Natural Foods
65,Dean Foods
66,Mohawk Industries
67,Franklin Resources


When you use the BETWEEN operator in SQL, do the results include the values of your endpoints, or not? Figure out the answer to this important question by writing a query that displays the order date and gloss_qty data for all orders where gloss_qty is between 24 and 29. Then look at your output to see if the BETWEEN operator included the begin and end values or not.


In [45]:
query = 'SELECT occurred_at, gloss_qty FROM orders WHERE gloss_qty BETWEEN 24 and 29;'
query_to_df(query)

Query ran for 0.6440184116363525 secs!


,occurred_at,gloss_qty
0,2016-02-01 19:27:27,29
1,2016-03-02 15:29:32,24
2,2016-10-14 23:54:21,28
3,2015-08-09 18:29:20,24
4,2016-02-01 20:00:37,26
...,...,...
479,2016-07-23 14:32:34,24
480,2014-06-26 04:12:30,24
481,2014-03-08 15:33:55,25
482,2014-03-09 07:21:16,27


Use the web_events table to find all information regarding individuals who were contacted via the organic or adwords channels, and started their account at any point in 2016, sorted from newest to oldest.

In [46]:
query = 'SELECT * FROM web_events WHERE channel IN ("organic", "adwords") \
        AND occurred_at between "2016-01-01" and "2017-01-01" ORDER BY occurred_at DESC;'

query_to_df(query)

Query ran for 1.7372901439666748 secs!


,id,account_id,occurred_at,channel
0,8493,4141,2016-12-31 16:31:23,organic
1,5661,1851,2016-12-31 06:55:38,organic
2,5562,1791,2016-12-31 02:08:50,adwords
3,7703,3351,2016-12-30 21:06:53,adwords
4,7921,3521,2016-12-30 20:15:48,organic
...,...,...,...,...
1020,7846,3471,2016-01-06 08:23:05,adwords
1021,6567,2541,2016-01-06 00:15:28,organic
1022,8775,4281,2016-01-04 22:51:01,adwords
1023,5581,1831,2016-01-02 21:27:39,organic


You will notice that using BETWEEN is tricky for dates! While BETWEEN is generally inclusive of endpoints, it assumes the time is at 00:00:00 (i.e. midnight) for dates. This is the reason why we set the right-side endpoint of the period at '2017-01-01'.


**The OR Operator**

* Similar to the `AND` operator, the `OR` operator can combine multiple statements.
* Each time you link a new statement with an `OR`, you will need to specify the column you are interested in looking at, just like with `AND`.
* You may link as many statements as you would like to consider at the same time.
* This operator works with all of the operations we have seen so far including arithmetic operators `(+, *, -, /)`, `LIKE`, `IN`, `NOT`, `AND`, and `BETWEEN` logic can all be linked together using the `OR` operator.
* When combining multiple of these operations, we frequently might need to use parentheses to ensure that logic we want to perform is being executed correctly.

<h3>Questions using the OR operator</h3>

Find list of orders ids where either gloss_qty or poster_qty is greater than 4000. Only include the id field in the resulting table.

In [47]:
query = 'SELECT id FROM orders WHERE gloss_qty > 4000 OR poster_qty > 4000;'

query_to_df(query)

Query ran for 0.04637908935546875 secs!


,id
0,362
1,731
2,1191
3,1913
4,1939
5,3778
6,3858
7,3963
8,4016
9,4230


Write a query that returns a list of orders where the standard_qty is zero and either the gloss_qty or poster_qty is over 1000.

In [48]:
query = 'SELECT * FROM orders WHERE standard_qty = 0 AND \
        (gloss_qty > 1000 OR poster_qty > 1000);'
query_to_df(query)

Query ran for 0.08092832565307617 secs!


,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,1913,2461,2013-12-29 09:50:38,0,6450,45,6495,0.0,48310.50,365.40,48675.90
1,4369,1111,2015-11-15 17:47:46,0,486,2988,3474,0.0,3640.14,24262.56,27902.70
2,4391,1161,2016-06-04 08:58:10,0,106,2967,3073,0.0,793.94,24092.04,24885.98
3,4420,1191,2016-05-21 23:21:14,0,43,1448,1491,0.0,322.07,11757.76,12079.83
4,4448,1231,2016-06-25 12:27:15,0,3178,23,3201,0.0,23803.22,186.76,23989.98
5,4698,1451,2015-02-26 06:13:21,0,484,4901,5385,0.0,3625.16,39796.12,43421.28
6,4942,1701,2015-09-24 21:02:25,0,10744,95,10839,0.0,80472.56,771.40,81243.96
7,5032,1831,2016-05-26 17:48:19,0,1448,287,1735,0.0,10845.52,2330.44,13175.96
8,5191,2051,2015-05-17 14:17:59,0,1041,0,1041,0.0,7797.09,0.00,7797.09
9,5791,2861,2014-10-24 12:06:22,0,10,11691,11701,0.0,74.90,94930.92,95005.82


Find all the company names that start with a 'C' or 'W', and the primary contact contains 'ana' or 'Ana', but it doesn't contain 'eana'.


In [49]:
query = 'SELECT * FROM accounts WHERE \
        (name LIKE "C%" OR name LIKE "W%") AND \
        (primary_poc LIKE "%ana%" or primary_poc LIKE "%Ana%") AND\
        primary_poc NOT LIKE "%eana%";'

query_to_df(query)

Query ran for 0.03810310363769531 secs!


,id,name,website,lats,longs,primary_poc,sales_rep_id
0,1061,CVS Health,www.cvshealth.com,41.467796,-73.767636,Anabel Haskell,321560
1,1361,Comcast,www.comcastcorporation.com,42.541548,-76.249924,Shana Sanborn,321650


In [50]:
# closing connection and cursor for the day
cursor.close()
cnxn.close()
print('All connections Closed!')

All connections Closed!
